# **FITS to HDF5**

In [1]:
# 3rd party # remove imports that are not needed
import numpy as np
import pandas as pd
import h5py
import astropy.table

In [2]:
data=astropy.table.Table.read('/data/des90.a/data/raulteix/data/BOSSDELVE/BOSSDELVEwpxs.fits', format='fits')

In [3]:
# creating mask
mask1 = (data['MAG_AUTO_G']!=99) * (data['MAG_AUTO_R']!=99) * (data['MAG_AUTO_I']!=99) * (data['MAG_AUTO_Z']!=99)
mask2 = (data['MAG_AUTO_G']!=99) * (data['MAG_AUTO_R']!=99) * (data['MAG_AUTO_I']!=99) 
mask3 = (data['MAG_AUTO_G']!=99) * (data['MAG_AUTO_R']!=99) * (data['MAG_AUTO_Z']!=99)
mask4 = (data['MAG_AUTO_G']!=99) * (data['MAG_AUTO_I']!=99) * (data['MAG_AUTO_Z']!=99)
mask5 = (data['MAG_AUTO_R']!=99) * (data['MAG_AUTO_I']!=99) * (data['MAG_AUTO_Z']!=99)

mask = mask1 + mask2 + mask3 + mask4 + mask5

In [11]:
bandlist = ['G', 'R', 'I', 'Z']
for band in bandlist:
    data[f'MAG_AUTO_{band}'][data[f'MAG_AUTO_{band}']==99]=-99
    data[f'MAGERR_AUTO_{band}'][data[f'MAG_AUTO_{band}']==99]=0

In [12]:
#creating the table to be converted
table = data[mask]['QUICK_OBJECT_ID', 'MAG_AUTO_G', 'MAG_AUTO_R', 'MAG_AUTO_I', 'MAG_AUTO_Z', 'MAGERR_AUTO_G', 'MAGERR_AUTO_R', 'MAGERR_AUTO_I', 'MAGERR_AUTO_Z'] #[tab_all['extended_class']>1]

#converting to pandas dataframe
dframe = pd.DataFrame(data=table['QUICK_OBJECT_ID'], columns = ['QUICK_OBJECT_ID'])
for label in table.columns:
        dframe[label] = table[label]

In [13]:
#creating hdf5 file from pandas dataframe
n = len(dframe) 

In [16]:
n

801489

In [14]:
np.sum(mask1)

441444

In [15]:
# 39 is just a particularly nice divisor of n in this case!
for i in range(39):
    dframe_i = dframe[(i*20551):(i+1)*20551]
    dframe_i.to_hdf(f'/data/des90.a/data/raulteix/data/BOSSDELVE/bpzparallelBOSSDELVEdata/input/DELVEBOSSmatched{i}.h5', key='df')